In [1]:
import pandas as pd
import numpy as np

import scipy.stats as ss
import statsmodels.api as sm
import statsmodels.formula.api as smf
from statsmodels.stats.api import anova_lm
from statsmodels.stats.multicomp import (pairwise_tukeyhsd,
                                         MultiComparison)
import pingouin as pg

import matplotlib.pyplot as plt
import seaborn as sns

In [94]:
orders = pd.read_csv('ab_orders.csv')
products = pd.read_csv('ab_products.csv')
usersdata = pd.read_csv('ab_users_data.csv')

<b> рассмотрим исходные таблицы, их состав, типы данных

In [3]:
orders

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,"{75, 22, 53, 84}"
1,1256,2022-08-26 00:02:21.000000,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27.000000,"{76, 34, 41, 38}"
3,1258,2022-08-26 00:02:56.000000,"{74, 6}"
4,1259,2022-08-26 00:03:37.000000,"{20, 45, 67, 26}"
...,...,...,...
4118,59422,2022-09-08 23:13:03.000000,"{84, 8, 24}"
4119,59439,2022-09-08 23:18:05.000000,"{9, 25, 75, 30, 6}"
4120,59464,2022-09-08 23:22:27.000000,"{60, 41, 46}"
4121,59487,2022-09-08 23:29:06.000000,"{9, 62, 77}"


In [4]:
orders.dtypes

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedCacheFailedWarning: Failed to use cache while checking for outdated package.
Set the environment variable OUTDATED_RAISE_EXCEPTION=1 for a full traceback.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs
/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package outdated is out of date. Your version is 0.2.1, the latest is 0.2.2.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


order_id          int64
creation_time    object
product_ids      object
dtype: object

In [5]:
orders.isna().sum()

/opt/tljh/user/lib/python3.7/site-packages/outdated/utils.py:18: OutdatedPackageWarning: The package pingouin is out of date. Your version is 0.4.0, the latest is 0.5.4.
Set the environment variable OUTDATED_IGNORE=1 to disable these warnings.
  **kwargs


order_id         0
creation_time    0
product_ids      0
dtype: int64

In [6]:
products

,product_id,name,price
0,1,сахар,150.0
1,2,чай зеленый в пакетиках,50.0
2,3,вода негазированная,80.4
3,4,леденцы,45.5
4,5,кофе 3 в 1,15.0
...,...,...,...
82,83,вафли,55.0
83,84,мандарины,90.4
84,85,варенье,200.3
85,86,кофе холодный,70.3


In [7]:
products.dtypes

product_id      int64
name           object
price         float64
dtype: object

In [8]:
products.isna().sum()

product_id    0
name          0
price         0
dtype: int64

In [9]:
usersdata

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37.000000,2022-08-26,1
...,...,...,...,...,...,...
4332,990,59422,create_order,2022-09-08 23:13:03.000000,2022-09-08,1
4333,1418,59439,create_order,2022-09-08 23:18:05.000000,2022-09-08,1
4334,1605,59464,create_order,2022-09-08 23:22:27.000000,2022-09-08,0
4335,1461,59487,create_order,2022-09-08 23:29:06.000000,2022-09-08,0


In [10]:
usersdata.dtypes

user_id      int64
order_id     int64
action      object
time        object
date        object
group        int64
dtype: object

In [11]:
usersdata.isna().sum()

user_id     0
order_id    0
action      0
time        0
date        0
group       0
dtype: int64

In [12]:
orders.order_id.nunique()

4123

In [13]:
usersdata.order_id.nunique()

4123

In [14]:
usersdata.user_id.nunique()

1017

In [15]:
usersdata_0 = usersdata.query('group==0')
usersdata_1 = usersdata.query('group==1')

In [16]:
usersdata_0.count()

user_id     1691
order_id    1691
action      1691
time        1691
date        1691
group       1691
dtype: int64

In [17]:
usersdata_1.count()

user_id     2646
order_id    2646
action      2646
time        2646
date        2646
group       2646
dtype: int64

In [18]:
pd.crosstab(usersdata.action, usersdata.group)

group,0,1
action,,
cancel_order,82,132
create_order,1609,2514


<b> проверим гипотезу о том, что система рекомендаций помогла изменить число отказов. для этого посчитаем p-value через хи-квадрат

In [19]:
from scipy.stats import chi2_contingency, chi2

In [20]:
stat, p, dof, expected = chi2_contingency(pd.crosstab(usersdata.action, usersdata.group))

In [21]:
stat, p

(0.018211165651942023, 0.8926523935841298)

<b> p-value больше 0.05, что не позволяет отклонить нулевую гипотезу. Смена алгоритма статистически значимо не повлияла на количество отказов

<b> теперь посчитаем, как повлияла смена алгоритма на средний чек

In [100]:
orders

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,"{75, 84, 53, 22}"
1,1256,2022-08-26 00:02:21.000000,"{56, 76, 39}"
2,1257,2022-08-26 00:02:27.000000,"{41, 34, 76, 38}"
3,1258,2022-08-26 00:02:56.000000,"{74, 6}"
4,1259,2022-08-26 00:03:37.000000,"{26, 67, 20, 45}"
...,...,...,...
4118,59422,2022-09-08 23:13:03.000000,"{8, 24, 84}"
4119,59439,2022-09-08 23:18:05.000000,"{6, 9, 75, 25, 30}"
4120,59464,2022-09-08 23:22:27.000000,"{41, 60, 46}"
4121,59487,2022-09-08 23:29:06.000000,"{9, 77, 62}"


<b> преобразуем множества в списки, чтобы сделать explode и сопоставить номера продуктов и посчитать стоимости заказов

In [113]:
orders['product_ids'] = orders.apply(lambda row: list(row['product_ids']), axis=1)

In [114]:
orders

,order_id,creation_time,product_ids
0,1255,2022-08-26 00:00:19.000000,"[75, 84, 53, 22]"
1,1256,2022-08-26 00:02:21.000000,"[56, 76, 39]"
2,1257,2022-08-26 00:02:27.000000,"[41, 34, 76, 38]"
3,1258,2022-08-26 00:02:56.000000,"[74, 6]"
4,1259,2022-08-26 00:03:37.000000,"[26, 67, 20, 45]"
...,...,...,...
4118,59422,2022-09-08 23:13:03.000000,"[8, 24, 84]"
4119,59439,2022-09-08 23:18:05.000000,"[6, 9, 75, 25, 30]"
4120,59464,2022-09-08 23:22:27.000000,"[41, 60, 46]"
4121,59487,2022-09-08 23:29:06.000000,"[9, 77, 62]"


In [115]:
orders1 = orders.explode('product_ids')

In [118]:
orders1.columns = ['order_id', 'creation_time', 'product_id']
orders1

,order_id,creation_time,product_id
0,1255,2022-08-26 00:00:19.000000,75
0,1255,2022-08-26 00:00:19.000000,84
0,1255,2022-08-26 00:00:19.000000,53
0,1255,2022-08-26 00:00:19.000000,22
1,1256,2022-08-26 00:02:21.000000,56
...,...,...,...
4121,59487,2022-09-08 23:29:06.000000,9
4121,59487,2022-09-08 23:29:06.000000,77
4121,59487,2022-09-08 23:29:06.000000,62
4122,59533,2022-09-08 23:41:24.000000,17


<b> сошьем таблицы и посчитаем стоимость каждого заказа

In [119]:
orders_big = orders1.merge(products, on='product_id', how = 'left')

In [120]:
orders_big

,order_id,creation_time,product_id,name,price
0,1255,2022-08-26 00:00:19.000000,75,сок ананасовый,120.0
1,1255,2022-08-26 00:00:19.000000,84,мандарины,90.4
2,1255,2022-08-26 00:00:19.000000,53,мука,78.3
3,1255,2022-08-26 00:00:19.000000,22,сок мультифрукт,120.0
4,1256,2022-08-26 00:02:21.000000,56,сосиски,150.0
...,...,...,...,...,...
13545,59487,2022-09-08 23:29:06.000000,9,чай черный листовой,83.5
13546,59487,2022-09-08 23:29:06.000000,77,курица,298.4
13547,59487,2022-09-08 23:29:06.000000,62,сок яблочный,120.0
13548,59533,2022-09-08 23:41:24.000000,17,морс брусничный,190.0


In [122]:
orders_prices = orders_big.groupby('order_id', as_index=False).agg({'price':'sum'})
orders_prices

,order_id,price
0,1255,408.7
1,1256,250.5
2,1257,310.2
3,1258,85.0
4,1259,228.0
...,...,...
4118,59422,241.2
4119,59439,518.5
4120,59464,185.9
4121,59487,501.9


<b> проверим корректность расчетов на примере заказа 1258

In [123]:
orders.query('order_id==1258')

,order_id,creation_time,product_ids
3,1258,2022-08-26 00:02:56.000000,"[74, 6]"


products.query('product_id==74')

In [126]:
products.query('product_id==6')

,product_id,name,price
5,6,сухарики,25.0


<b> всё корректно сложилось. <br> теперь сделаем таблицу с группами старого и нового алгоритма и суммами по заказам

In [127]:
usersdata_create = usersdata.query('action=="create_order"')

In [128]:
usersdata_create

,user_id,order_id,action,time,date,group
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0
4,967,1259,create_order,2022-08-26 00:03:37.000000,2022-08-26,1
...,...,...,...,...,...,...
4332,990,59422,create_order,2022-09-08 23:13:03.000000,2022-09-08,1
4333,1418,59439,create_order,2022-09-08 23:18:05.000000,2022-09-08,1
4334,1605,59464,create_order,2022-09-08 23:22:27.000000,2022-09-08,0
4335,1461,59487,create_order,2022-09-08 23:29:06.000000,2022-09-08,0


In [129]:
usersdata_create = usersdata_create.merge(orders_prices, on='order_id', how = 'left')

In [130]:
usersdata_create

,user_id,order_id,action,time,date,group,price
0,964,1255,create_order,2022-08-26 00:00:19.000000,2022-08-26,0,408.7
1,965,1256,create_order,2022-08-26 00:02:21.000000,2022-08-26,1,250.5
2,964,1257,create_order,2022-08-26 00:02:27.000000,2022-08-26,0,310.2
3,966,1258,create_order,2022-08-26 00:02:56.000000,2022-08-26,0,85.0
4,967,1259,create_order,2022-08-26 00:03:37.000000,2022-08-26,1,228.0
...,...,...,...,...,...,...,...
4118,990,59422,create_order,2022-09-08 23:13:03.000000,2022-09-08,1,241.2
4119,1418,59439,create_order,2022-09-08 23:18:05.000000,2022-09-08,1,518.5
4120,1605,59464,create_order,2022-09-08 23:22:27.000000,2022-09-08,0,185.9
4121,1461,59487,create_order,2022-09-08 23:29:06.000000,2022-09-08,0,501.9


In [134]:
usersdata_create_0 = usersdata_create.query('group==0').price
usersdata_create_1 = usersdata_create.query('group==1').price

In [136]:
usersdata_create_0.mean()

374.4510254816656

In [137]:
usersdata_create_1.mean()

361.9766507557677

<b> для двух выборок проведем t-тест. Нулевая гипотеза - введение алгоритма не повлияло на выборку по чекам

In [138]:
ss.ttest_ind(usersdata_create_0, usersdata_create_1)

Ttest_indResult(statistic=1.6276451303367758, pvalue=0.10367659971189314)

<b> Как видим, p-value больше 0.05, что говорит о том, что смена алгоритма статистически значимо не повлияла на средний чек. <br> Проверим результат через дисперсионный анализ для двух групп

In [139]:
pg.anova(data=usersdata_create, dv="price", between="group")

,Source,ddof1,ddof2,F,p-unc,np2
0,group,1,4121,2.649229,0.103677,0.000642


<b> Значение p-value аналогично тому, что было получено при выполнении t-теста. <br> Таким образом, новый алгоритм никак не влияет на средний чек и количество отказов по заказам. <br>Смена алгоритма не принесет никакой экономической выгоды компании